In [1]:
import tensorflow as tf

#### 1.定义变量以及滑动平均类

In [2]:
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

# 定义一个滑动平均类class。初始化时给定衰减率(0.99)和控制衰减变量step
ema = tf.train.ExponentialMovingAverage(0.99, step)
# 定义一个更新变量滑动平均的操作。这里需要定义一个列表，每次执行这个操作时
# 这个列表中的变量就会被更新
maintain_averages_op = ema.apply([v1])

#### 2.查看不同迭代中变量取值的变化

In [4]:
config = tf.ConfigProto(allow_soft_placement=True)
with tf.Session(config=config) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 通过ema.average(v1)获取滑动平均之后的变量取值
    # 在初始化变量v1的值和v1的滑动平均都是0
    print(sess.run([v1, ema.average(v1)])) # 输出[0.0, 0.0]
    
    # 更新变量v1的值到5
    sess.run(tf.assign(v1, 5))
    # 更新变量v1的滑动平均值。衰减率为min{0.99, (1+step)/(10+step) = 0.1} = 0.1
    # 所以v1的滑动平均会被更新为0.1x0+0.9x5=4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)])) # 输出[5, 4.5]
    
    # 更新step的值为10000
    sess.run(tf.assign(step, 10000))
    # 更新v1的值为10
    sess.run(tf.assign(v1, 10))
    # 更新变量v1的滑动平均值。衰减率为min{0.99, (1+step)/(10+step) = 0.9999} = 0.99
    # 所有v1的滑动平均值为0.99*4.5+0.01*10=4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)])) # 输出[10.0, 4.554998]
    
    # 再次更新滑动平均值，得到的新滑动平均值为0.99*4.55+0.01*10=4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1, ema.average(v1)])) # 输出[10.0, 4.60945]

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
[10.0, 4.6094499]
